# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [103]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [104]:
# code from Firas - supresses warning alerts
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
from watermark import watermark
print(watermark())
print(watermark(iversions=True, globals_=globals(), packages=""))

Last updated: 2023-05-27T20:46:42.440714-07:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 154 Stepping 4, GenuineIntel
CPU cores   : 12
Architecture: 64bit

hvplot: 0.8.0
pandas: 1.3.5



In [105]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [106]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [108]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [109]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [110]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [111]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1,11))

In [112]:
# Create an empy list to store the inertia values
inertia = []

In [114]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model_k = KMeans(n_clusters=i, random_state=0)
    model_k.fit(df_market_data_scaled)
    inertia.append(model_k.inertia_)

In [117]:
# Create a dictionary with the data to plot the Elbow curve

# `ec` abbreviates 'Elbow curve'

ec_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve

df_ec_data = pd.DataFrame(ec_data)
df_ec_data.head(10)

ValueError: All arrays must be of the same length

In [118]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
plot_elbow_line = df_ec_data.hvplot.line(
    x="k",
    y= "inertia",
    title="Elbow Curve",
    xticks=k
)

plot_elbow_line

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** [4] is best for `k` based on visual made with `hvplot.line`
+ [k=4] is the point along the x-axis where the line curve flattens 
            

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [119]:
# Initialize the K-Means model using the best value for k
model_k = KMeans(n_clusters=4)

In [120]:
# Fit the K-Means model using the scaled data
model_k.fit(df_market_data_scaled)

KMeans(n_clusters=4)

In [121]:
# Predict the clusters to group the cryptocurrencies using the scaled data

predicted_cluster_values = model_k.predict(df_market_data_scaled)

# View the resulting array of cluster values.

# clusters = (predicted_cluster_values) #### <--- this code works to define 'Clusters' variable, however...

Cluster = (predicted_cluster_values+1) # modified with a (+1) to each value in the array - improve visual of the hvplot.scatter output

# Cluster # code adds a +1 to each value, so the 4 clusters will be labeled as (1-4) instead of (0-3)


In [123]:
# Create a copy of the DataFrame

df_price_chg_clusters = df_market_data_scaled.copy()

In [124]:
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [125]:
# Add a new column to the DataFrame with the predicted clusters
df_price_chg_clusters['Cluster'] = Cluster

# Display sample data
df_price_chg_clusters.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,4
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,4
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,4


In [126]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

plot_cluster = df_price_chg_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols="coin_id")

plot_cluster


:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)


from `pandas.pydata.org`, found this: [pandas.DataFrame.plot.scatter.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.scatter.html)



then also discovered more options from `matplotlib.org` to use gradient color scales ['colormap'](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [127]:
## code below is alternative hvplot.scatter greyscale compatible, 'copper' color - it does not require any additional imports to starter code

In [128]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

copper_plot_cluster = df_price_chg_clusters.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    c=Cluster, 
    title="Crypto Clusters - Price Change Percentage (Day, Week)",
    colormap='copper',
    hover_cols="coin_id")

copper_plot_cluster

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,_color,coin_id)

In [129]:
pca_copper_cluster = df_pca_predict_clusters.hvplot.scatter(x="PC1", y="PC2", c="PCA_Cluster", title="PCA Crypto Cluster Plot", colormap='copper', hover_cols="coin_id")


### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [130]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [131]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_scaled_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
market_data_scaled_pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [132]:
# Retrieve the explained variance to determine how much information 

pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

In [133]:
# the total amount of variance or percentage that can be attributed to each principal component breaks down as follows

var_ex = sum(pca.explained_variance_ratio_)

display(var_ex)

0.8950316570309841

In [134]:
##      Components of total variance identified:
###     x="price_change_percentage_24h"
###     y="price_change_percentage_7d"
###     c=Cluster

###      PCA model instance created and set to: `n_components=3`
Three components of the total variance from the PCA of 'model_k'
1) ***price_change_percentage_24h***
2) ***price_change_percentage_7d***
3) a variable array defined as ***`Cluster`*** with a capital ***"C"***



#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

In [135]:
total_explained_variance = 100*var_ex 

print(f"The total explained variance is: {total_explained_variance: ,.2f}%")


The total explained variance is:  89.50%


**Answer:** # ^^^ see above ^^^

In [136]:
# original data frame from 'pd.read_csv' is defined as "df_market_data"

# "df_market_data_scaled" is the original scaled data frame

# copy of "df_market_data_scaled" is a variable array defined as "Cluster" in the PCA model where n=3

In [137]:
# Creating a DataFrame with the PCA data

df_principal_comp = pd.DataFrame(
    market_data_scaled_pca_data,
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original data
df_principal_comp["coin_id"] = df_market_data.index

# Set the coinid column as index
df_principal_comp = df_principal_comp.set_index("coin_id")

# Display sample data
df_principal_comp.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [138]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))


In [139]:
# Create an empy list to store the inertia values
inertia = []

In [140]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model_ki = KMeans(n_clusters=i, random_state=0) # create kmeans model
    model_ki.fit(df_market_data_scaled) # fit model to data
    inertia.append(model_ki.inertia_) # append

In [141]:
# Create a DataFrame with the data to plot the Elbow curve

ec_data_i = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve

df_ec_data_i = pd.DataFrame(ec_data_i)

In [142]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

plot_elbow_pca = df_ec_data_i.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve - i", 
    xticks=k
)

In [144]:
# Create a DataFrame with the data to plot the Elbow curve

df_ec_data_i = pd.DataFrame(ec_data_i)

ec_data_i = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve

df_ec_data_i = pd.DataFrame(ec_data_i)
df_ec_data_i.head(10)

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.302379
5,6,52.888518
6,7,43.914690
7,8,37.517032
8,9,32.485241
9,10,28.222899


#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** # 4 is still best answer


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** # no

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [145]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [146]:
# Fit the K-Means model using the PCA data
model.fit(df_principal_comp)

KMeans(n_clusters=4, random_state=0)

In [147]:
df_principal_comp[:10]

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715
binancecoin,-0.516534,1.388377,0.804071
chainlink,-0.450711,0.517699,2.846143
cardano,-0.345600,0.729439,1.478013
litecoin,-0.649468,0.432165,0.600303


In [148]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_clusters = model.predict(df_principal_comp)

# View the resulting array of cluster values.
PCA_Cluster = (pca_clusters+1)

In [149]:
# Create a copy of the DataFrame with the PCA data
df_pca_predict_clusters = df_principal_comp.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_predict_clusters["PCA_Cluster"] = PCA_Cluster

# Display sample data
df_pca_predict_clusters

,PC1,PC2,PC3,PCA_Cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,1
ethereum,-0.458261,0.458466,0.952877,1
tether,-0.433070,-0.168126,-0.641752,4
ripple,-0.471835,-0.222660,-0.479053,4
bitcoin-cash,-1.157800,2.041209,1.859715,1
binancecoin,-0.516534,1.388377,0.804071,1
chainlink,-0.450711,0.517699,2.846143,1
cardano,-0.345600,0.729439,1.478013,1
litecoin,-0.649468,0.432165,0.600303,1


In [150]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_plot_cluster = df_pca_predict_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="PCA_Cluster",
    hover_cols=["coin_id"],
    title="PCA Crypto Cluster Plot"
    )

pca_plot_cluster

:NdOverlay   [PCA_Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

In [151]:
pca_copper_cluster = df_pca_predict_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    c="PCA_Cluster",
    colormap='copper',
    hover_cols=["coin_id"],
    title="PCA Crypto Cluster Plot"
    )

pca_copper_cluster

:Scatter   [PC1]   (PC2,PCA_Cluster,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [152]:
# Composite plot to contrast the Elbow curves
plot_elbow_line * plot_elbow_pca

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [153]:
# Compoosite plot to contrast the clusters
plot_cluster + pca_plot_cluster 

:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [PCA_Cluster]
      :Scatter   [PC1]   (PC2,coin_id)

In [154]:
copper_plot_cluster + pca_copper_cluster

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,_color,coin_id)
   .Scatter.II :Scatter   [PC1]   (PC2,PCA_Cluster,coin_id)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** # fewer features result in tighter grouping of data plots making an easier visual id of cluster count